In [1]:
import pandas as pd
import numpy as np
import pickle
from typing import Optional, Union, Tuple
from cmapPy.pandasGEXpress.parse import parse
import numpy as np
import json
from collections import Counter
import sys
from numpy.core.multiarray import ndarray
import sys
import csv 
import requests 
import xml.etree.ElementTree as ET 
from xml.dom import minidom

/Users/dweepa/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Parsing the XML file

## Importing drugbank dataset

In [2]:
drugbank = pd.read_csv("drugbank.csv",low_memory=False)
drugbank = drugbank.dropna(how='all', axis='columns')
drugbank.head(2)

,Unnamed: 0,atc-level1,atc-level2,atc-level3,atc-level4,atc-level5,inchi,inchi_key,name,pubchem,...,syn59,syn6,syn60,syn61,syn62,syn63,syn64,syn7,syn8,syn9
0,0,B,B01,B01A,B01AE,B01AE02,NaN,NaN,Lepirudin,46507011.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,L,L01,L01X,L01XC,L01XC06,NaN,NaN,Cetuximab,46507042.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
p = pd.read_csv("../Data/GSE70138_Broad_LINCS_pert_info.txt",delimiter="\t")
p.canonical_smiles = p.canonical_smiles.str.upper()
p.head(2)

,pert_id,canonical_smiles,inchi_key,pert_iname,pert_type
0,BRD-K70792160,CCN(CC)CCCCN1C2CCCCC2OC2CCC(CL)CC12,GYBXAGDWMCJZJK-UHFFFAOYSA-N,10-DEBC,trt_cp
1,BRD-K68552125,CCCCCCCCCCCCCC(=O)O[C@@H]1[C@@H](C)[C@]2(O)[C@...,PHEDXBVPIONUQT-RGYGYFBISA-N,phorbol-myristate-acetate,trt_cp


In [4]:
X = pd.read_pickle("../Data/full")

In [5]:
# two different smiles for the same inchi_key
print(drugbank[drugbank.inchi_key=='QHMWCHQXCUNUAK-UHFFFAOYSA-N'].smiles.tolist()[0])
print(p[p.inchi_key=='QHMWCHQXCUNUAK-UHFFFAOYSA-N'].canonical_smiles.tolist()[0])

FC1=CC=C(C=C1)C(N1C=CN=C1)(C1=CC=CC=C1)C1=CC=CC=C1F
FC1CCC(CC1)C(C1CCCCC1)(C1CCCCC1F)N1CCNC1


## Get the atc codes via drugbank

## Import the final dataset

In [36]:
# pd.DataFrame(li).to_csv("atc-codes.csv",index=False)
atc = pd.read_csv("atc-codes.csv")
atc.name=atc.name.str.lower()
atc.head(2)

,Unnamed: 0,inchi_key,level1,level2,level3,level4,level5,name,smiles
0,0,IETKPTYAGKZLKY-UHFFFAOYSA-N,NaN,NaN,NaN,NaN,NaN,balaglitazone,CN1C(COC2CCC(CC3SC(=O)NC3=O)CC2)NC2CCCCC2C1=O
1,1,QHMWCHQXCUNUAK-UHFFFAOYSA-N,G,G01,G01A,G01AF,G01AF18,flutrimazole,FC1CCC(CC1)C(C1CCCCC1)(C1CCCCC1F)N1CCNC1


In [7]:
atc.isnull().sum()

Unnamed: 0       0
inchi_key        0
level1        1484
level2        1484
level3        1484
level4        1484
level5        1484
name          1069
smiles           0
dtype: int64

## Import original ATC standalone dataset

In [8]:
atc_data = pd.read_excel("atc.xlsx")
atc_data.head(2)

,Class ID,Preferred Label,Synonyms,Definitions,Obsolete,CUI,Semantic Types,Parents,ATC LEVEL,Is Drug Class,Semantic type UMLS property
0,http://purl.bioontology.org/ontology/UATC/A03AX13,silicones,NaN,NaN,False,C0037114,http://purl.bioontology.org/ontology/STY/T109|...,http://purl.bioontology.org/ontology/UATC/A03AX,5.0,NaN,http://purl.bioontology.org/ontology/STY/T109|...
1,http://purl.bioontology.org/ontology/UATC/J01DB07,cefatrizine,NaN,NaN,False,C0007545,http://purl.bioontology.org/ontology/STY/T195|...,http://purl.bioontology.org/ontology/UATC/J01DB,5.0,NaN,http://purl.bioontology.org/ontology/STY/T195|...


## Get pert_iname from pert_id

In [9]:
id_to_name = dict()
name_to_id = dict()
tar = np.unique(X.target)
for i in range(len(tar)):
    a = p[p.pert_id==tar[i]].pert_iname
    if(a.to_string().split()[1]==''):
        a = a.to_string().split()[0]
    else:
        a = a.to_string().split()[1]
    a=a.translate({ord(')'): None})
    a=a.translate({ord(']'): None})
#     sys.stdout.write('\r%d/%d\t\t%s' % (i, len(tar),a))
    print('%d/%d\t\t%s' % (i, len(tar),a))
    id_to_name[tar[i]]=a
    name_to_id[a]=tar[i]

0/2170		balaglitazone
1/2170		flutrimazole
2/2170		KBG
3/2170		biperiden
4/2170		drospirenone
5/2170		bupivacaine
6/2170		garcinol
7/2170		naftopidil
8/2170		nitrendipine
9/2170		ethinyl-estradiol
10/2170		meptazinol
11/2170		lofexidine
12/2170		alizapride
13/2170		mepivacaine
14/2170		fulvestrant
15/2170		XL-888
16/2170		metoprolol
17/2170		maraviroc
18/2170		chlorphenamine
19/2170		bupropion
20/2170		lisofylline
21/2170		BAY-K8644
22/2170		telmesteine
23/2170		docetaxel
24/2170		CPI-613
25/2170		fusidic-acid
26/2170		esmolol
27/2170		labetalol
28/2170		bopindolol
29/2170		isofloxythepin
30/2170		kakonein
31/2170		galeterone
32/2170		racecadotril
33/2170		doxycycline
34/2170		norethindrone
35/2170		flecainide
36/2170		verapamil
37/2170		argatroban
38/2170		tipranavir
39/2170		propranolol
40/2170		GSK-429286A
41/2170		imiloxan
42/2170		granisetron
43/2170		carvedilol
44/2170		pravastatin
45/2170		beclomethasone-dipropionate
46/2170		lorazepam
47/2170		cefepime
48/2170		nutlin-3
49/2170

433/2170		roquinimex
434/2170		cobimetinib
435/2170		azacitidine
436/2170		foretinib
437/2170		ixabepilone
438/2170		rottlerin
439/2170		penitrem-a
440/2170		raclopride
441/2170		nepafenac
442/2170		oxybuprocaine
443/2170		favipiravir
444/2170		AGI-6780
445/2170		VE-822
446/2170		clemizole
447/2170		SAR-245408
448/2170		ciprofloxacin
449/2170		GSK-1904529A
450/2170		LDN-193189
451/2170		AKT-inhibitor-1-2
452/2170		nebivolol
453/2170		LY-2334737
454/2170		PD-184352
455/2170		luteolin
456/2170		nicotine
457/2170		ledipasvir
458/2170		dronedarone
459/2170		raltegravir
460/2170		dicloxacillin
461/2170		PSI-7976
462/2170		AZ-628
463/2170		XMD-1499
464/2170		fluconazole
465/2170		rilpivirine
466/2170		UK-383367
467/2170		tizanidine
468/2170		pramipexole
469/2170		voriconazole
470/2170		HG-5-88-01
471/2170		nafamostat
472/2170		Gossypetin
473/2170		isbufylline
474/2170		cabazitaxel
475/2170		deferiprone
476/2170		tebipenem
477/2170		minoxidil
478/2170		BMS-599626
479/2170		fiacitabine
480/217

864/2170		levonorgestrel
865/2170		ciclesonide
866/2170		dacarbazine
867/2170		yohimbine
868/2170		danegaptide
869/2170		SR-27897
870/2170		gimeracil
871/2170		rotundine
872/2170		liranaftate
873/2170		niclosamide
874/2170		asymmetrical-dimethylarginine
875/2170		esculin
876/2170		tofacitinib
877/2170		NF-449
878/2170		I-BET-151
879/2170		dolutegravir
880/2170		bindarit
881/2170		tamibarotene
882/2170		omecamtiv-mecarbil
883/2170		setiptiline
884/2170		GSK-1070916
885/2170		hydroflumethiazide
886/2170		glibenclamide
887/2170		tonabersat
888/2170		trientine
889/2170		clozapine
890/2170		efaproxiral
891/2170		conivaptan
892/2170		idebenone
893/2170		zardaverine
894/2170		GSK-256066
895/2170		AZD-8330
896/2170		doxepin
897/2170		ABT-724
898/2170		SB-525334
899/2170		PF-03758309
900/2170		etoposide
901/2170		3,3'-diindolylmethane
902/2170		tirofiban
903/2170		camptothecin
904/2170		paroxetine
905/2170		clonazepam
906/2170		bumetanide
907/2170		narlaprevir
908/2170		imipramine
909/2170		cil

1317/2170		lisinopril
1318/2170		MK-2206
1319/2170		palmitoylethanolamide
1320/2170		CGS-20625
1321/2170		terbinafine
1322/2170		epothilone-b
1323/2170		torin-2
1324/2170		ALW-II-38-3
1325/2170		etizolam
1326/2170		lacosamide
1327/2170		tyrphostin-AG-1478
1328/2170		amsacrine
1329/2170		ZK-93426
1330/2170		KIN001-055
1331/2170		ENMD-2076
1332/2170		SB-743921
1333/2170		BRD-K68548958
1334/2170		phorbol-myristate-acetate
1335/2170		lidamidine
1336/2170		PF-03814735
1337/2170		FR-180204
1338/2170		quetiapine
1339/2170		flumatinib
1340/2170		GDC-0152
1341/2170		golvatinib
1342/2170		sertindole
1343/2170		diacerein
1344/2170		lumacaftor
1345/2170		afloqualone
1346/2170		tacrolimus
1347/2170		mestinon
1348/2170		atorvastatin
1349/2170		mericitabine
1350/2170		volasertib
1351/2170		AZD-8055
1352/2170		ibrutinib
1353/2170		escitalopram
1354/2170		amantadine
1355/2170		aripiprazole
1356/2170		erlotinib
1357/2170		BAY-87-2243
1358/2170		sertraline
1359/2170		rimcazole
1360/2170		ranitidine
1361/

1772/2170		lopinavir
1773/2170		genz-644282
1774/2170		goserelin-acetate
1775/2170		PF-562271
1776/2170		tanaproget
1777/2170		motesanib
1778/2170		linifanib
1779/2170		hexachlorophene
1780/2170		bosutinib
1781/2170		reboxetine
1782/2170		verteporfin
1783/2170		nemonapride
1784/2170		levcromakalim
1785/2170		cevimeline
1786/2170		saracatinib
1787/2170		XMD11-85H
1788/2170		GSK-1059615
1789/2170		THZ-2-98-01
1790/2170		HG-6-64-01
1791/2170		WH-4-025
1792/2170		WZ-4-145
1793/2170		XMD-16144
1794/2170		BI-2536
1795/2170		AZD-7762
1796/2170		EGFP
1797/2170		EGFP
1798/2170		EGFP
1799/2170		PIK3CA
1800/2170		EGFP
1801/2170		EGFP
1802/2170		EGFP
1803/2170		EGFP
1804/2170		EGFP
1805/2170		EGFP
1806/2170		EGFP
1807/2170		PIK3CA
1808/2170		EGFP
1809/2170		EGFP
1810/2170		PIK3CA
1811/2170		PIK3CA
1812/2170		TP53
1813/2170		PIK3CA
1814/2170		BCL2L1
1815/2170		PIK3CA
1816/2170		PIK3CA
1817/2170		KRAS
1818/2170		JUN
1819/2170		AXL
1820/2170		JUN
1821/2170		HDAC1
1822/2170		HDAC2
1823/2170		HDAC3
182

In [10]:
print(len(name_to_id))
len(id_to_name)

1826


2170

## Get ATC codes from OG data

In [11]:
# Unnamed: 0	inchi_key	level1	level2	level3	level4	level5	name	smiles
li1=[]
count=0
for i in name_to_id.keys():
    if(i in list(atc_data['Preferred Label'])):\
        count+=1
    atcdict = dict()
    try:
        atc_class = atc_data[atc_data['Preferred Label']==i]['Class ID'].tolist()[0].split('/')[-1]
        print("%s\t%s\t%s"%(i , name_to_id[i], atc_class))
        atcdict["name"] = i
        atcdict["pert_id"] = name_to_id[i]
        atcdict["level1"]=atc_class[0]
        atcdict["level2"]=atc_class[0:2]
        atcdict["level3"]=atc_class[0:3]
        atcdict["level4"]=atc_class[0:5]
        atcdict["level5"]=atc_class
        atcdict['inchi_key']=''
        atcdict['smiles']=''
        li1.append(atcdict)
    except Exception as e:
        pass

flutrimazole	BRD-A00218260	D01AC16
biperiden	BRD-A00546892	N04AA02
drospirenone	BRD-A00938334	G03AC10
bupivacaine	BRD-A01636364	N01BB01
nitrendipine	BRD-A02006392	C08CA08
meptazinol	BRD-A02710418	N02AX05
lofexidine	BRD-A02990301	N07BC04
alizapride	BRD-A03061970	A03FA05
mepivacaine	BRD-A03216249	N01BB03
fulvestrant	BRD-A03249105	L02BA03
metoprolol	BRD-A03623303	C07AB02
maraviroc	BRD-A04352665	J05AX09
chlorphenamine	BRD-A04553218	R06AB04
bupropion	BRD-A05186015	N06AX12
docetaxel	BRD-A05821830	L01CD02
esmolol	BRD-A07395371	C07AB09
labetalol	BRD-A07440155	C07AG01
bopindolol	BRD-A07563059	C07AA17
racecadotril	BRD-A08187463	A07XA04
doxycycline	BRD-A08545410	A01AB22
flecainide	BRD-A09472452	C01BC04
verapamil	BRD-A09533288	C08DA01
argatroban	BRD-A10012892	B01AE03
tipranavir	BRD-A10039652	J05AE09
propranolol	BRD-A10070317	C07AA05
granisetron	BRD-A10967948	A04AA02
carvedilol	BRD-A10977446	C07AG02
pravastatin	BRD-K31447894	C10AA03
lorazepam	BRD-A11990600	N05BA06
cefepime	BRD-A12077521	J01DE01
cis

dronedarone	BRD-K05524748	C01BD07
raltegravir	BRD-K05658747	J05AX08
dicloxacillin	BRD-K05673000	J01CF01
fluconazole	BRD-K05977355	D01AC15
rilpivirine	BRD-K06240250	J05AG05
tizanidine	BRD-K06335600	M03BX02
pramipexole	BRD-K06388322	N04BC05
voriconazole	BRD-K06557128	J02AC03
cabazitaxel	BRD-K06858286	L01CD04
deferiprone	BRD-K06878038	V03AC02
minoxidil	BRD-K06902185	D11AX01
trimethoprim	BRD-K07208025	J01EA01
cinnarizine	BRD-K07220430	N07CA02
moclobemide	BRD-K07237224	N06AG02
alvimopan	BRD-K07857022	A06AH02
entecavir	BRD-K08206212	J05AF10
diclofenac	BRD-K08252256	D11AX18
irinotecan	BRD-K08547377	L01XX19
palonosetron	BRD-K08924299	A04AA05
clioquinol	BRD-K09255212	P01AA02
levofloxacin	BRD-K09471561	J01MA12
mibefradil	BRD-K09549677	C08CX01
dabrafenib	BRD-K09951645	L01XE23
sulfasalazine	BRD-K10670311	A07EC01
phenylbutazone	BRD-K10843433	M02AA01
tolcapone	BRD-K10852020	N04BX01
latanoprost	BRD-K10961822	S01EE01
gemfibrozil	BRD-K11129031	C10AB04
norfloxacin	BRD-K11196887	S01AE02
alectinib	BRD-K11

phenytoin	BRD-K55930204	N03AB02
oxiconazole	BRD-K56104152	G01AF17
vemurafenib	BRD-K56343971	L01XE15
ambroxol	BRD-K56558538	R05CB06
clofazimine	BRD-K56614220	J04BA01
fomepizole	BRD-K56810756	V03AB34
epirubicin	BRD-K56844688	L01DB03
enzalutamide	BRD-K56851771	L02BB04
indometacin	BRD-K57222227	C01EB03
fidaxomicin	BRD-K57371763	A07AA12
tiapride	BRD-K57432881	N05AL03
enalapril	BRD-K57545991	C09AA02
pentoxifylline	BRD-K57569181	C04AD03
cariprazine	BRD-K57764956	N05AX15
vilazodone	BRD-K58010567	N06AX24
rasagiline	BRD-K58114536	N04BD02
roxithromycin	BRD-K58211978	J01FA06
dexfenfluramine	BRD-K58626373	A08AA04
chlorprothixene	BRD-K59058766	N05AF03
naproxen	BRD-K59197931	G02CC02
propentofylline	BRD-K59273480	N06BC02
voglibose	BRD-K59331299	A10BF03
linopirdine	BRD-K59332007	N06BX09
tedizolid	BRD-K59436580	J01XX11
maribavir	BRD-K59524082	J05AX10
quinidine	BRD-K59632282	C01BA01
pasireotide	BRD-K59715032	H01CB05
irbesartan	BRD-K60038276	C09CA04
tiagabine	BRD-K60160658	N03AG06
paricalcitol	BRD-K605850

pimecrolimus	BRD-K97729603	D11AH02
chlorpropamide	BRD-K97746869	A10BB02
theophylline	BRD-K97799481	R03DA04
rupatadine	BRD-K98004573	R06AX28
clonidine	BRD-K98530306	C02AC01
flumazenil	BRD-K98769987	V03AB25
felbamate	BRD-K99107520	N03AX10
quinine	BRD-K99257182	P01BC01
hexachlorophene	BRD-K99792991	D08AE01
bosutinib	BRD-K99964838	L01XE14
reboxetine	BRD-M39350793	N06AX18
verteporfin	BRD-M45964048	S01LA01
cevimeline	BRD-M98279124	N07AX03


In [12]:
# 1 Instance of dict
li1[0]

{'inchi_key': '',
 'level1': 'D',
 'level2': 'D0',
 'level3': 'D01',
 'level4': 'D01AC',
 'level5': 'D01AC16',
 'name': 'flutrimazole',
 'pert_id': 'BRD-A00218260',
 'smiles': ''}

In [51]:
# atc = atc.drop(['Unnamed: 0'],axis=1)
atc['pert_id'] = atc.name
atc.pert_id = atc.pert_id.replace(name_to_id) 
atc.isnull().sum()
atc.head(2)

,Unnamed: 0,inchi_key,level1,level2,level3,level4,level5,name,smiles,pert_id
0,0,IETKPTYAGKZLKY-UHFFFAOYSA-N,NaN,NaN,NaN,NaN,NaN,balaglitazone,CN1C(COC2CCC(CC3SC(=O)NC3=O)CC2)NC2CCCCC2C1=O,BRD-A00147595
1,1,QHMWCHQXCUNUAK-UHFFFAOYSA-N,G,G01,G01A,G01AF,G01AF18,flutrimazole,FC1CCC(CC1)C(C1CCCCC1)(C1CCCCC1F)N1CCNC1,BRD-A00218260


In [52]:
newatc = pd.DataFrame(li1)
newatc.isnull().sum()

inchi_key    0
level1       0
level2       0
level3       0
level4       0
level5       0
name         0
pert_id      0
smiles       0
dtype: int64

# Append Data

In [53]:
wholeatc = newatc.append(atc)
wholeatc.isnull().sum()

Unnamed: 0     768
inchi_key        0
level1        1484
level2        1484
level3        1484
level4        1484
level5        1484
name          1069
pert_id       1069
smiles           0
dtype: int64

In [54]:
for i in Counter(wholeatc.name):
    if(Counter(wholeatc.name)[i]>1):
        print(i,Counter(wholeatc.name)[i])
len(wholeatc.name)

flutrimazole 2
biperiden 2
bupivacaine 2
nitrendipine 2
meptazinol 2
lofexidine 2
alizapride 2
mepivacaine 2
metoprolol 2
chlorphenamine 2
bupropion 2
esmolol 2
labetalol 2
bopindolol 2
racecadotril 2
flecainide 2
verapamil 2
propranolol 2
carvedilol 2
lorazepam 2
pomalidomide 2
troglitazone 2
doxazosin 2
alaproclate 2
milnacipran 2
rebamipide 2
indobufen 2
butoconazole 2
bambuterol 2
ibuprofen 2
lenalidomide 2
phensuximide 2
trimipramine 2
perhexiline 2
mianserin 2
ondansetron 2
mecamylamine 2
chlormezanone 2
praziquantel 2
amlodipine 2
benfluorex 2
pantoprazole 2
itraconazole 2
ifenprodil 2
ofloxacin 2
warfarin 2
aminoglutethimide 2
everolimus 2
propafenone 2
nicardipine 2
tolperisone 2
acebutolol 2
bicalutamide 2
disopyramide 2
felodipine 2
fluoxetine 2
methocarbamol 2
tioconazole 2
sotalol 2
etoposide 2
fenticonazole 2
metixene 2
suprofen 2
ibrutinib 2
rabeprazole 2
ketorolac 2
gemifloxacin 2
ornidazole 2
pinacidil 2
indoprofen 2
pentobarbital 2
proglumide 2
doxapram 2
citalopram 2

2938

In [55]:
len(atc.name) + len(newatc.name)

2938

In [56]:
print(atc[atc.name=='saracatinib'])
print('\n',newatc[newatc.name=='saracatinib'],'\n')
print(wholeatc[wholeatc.name=='saracatinib'])

      Unnamed: 0                    inchi_key level1 level2 level3 level4  \
651          651  OUKYUETWWIPKQR-UHFFFAOYSA-N    NaN    NaN    NaN    NaN   
1786        1786  OUKYUETWWIPKQR-UHFFFAOYSA-N    NaN    NaN    NaN    NaN   

     level5         name                                             smiles  \
651     NaN  saracatinib  CN1CCN(CCOC2CC(OC3CCOCC3)C3C(NC4C5OCOC5CCC4CL)...   
1786    NaN  saracatinib  CN1CCN(CC1)CCOC2=CC(=C3C(=C2)N=CN=C3NC4=C(C=CC...   

            pert_id  
651   BRD-U07805514  
1786  BRD-U07805514  

 Empty DataFrame
Columns: [inchi_key, level1, level2, level3, level4, level5, name, pert_id, smiles]
Index: [] 

      Unnamed: 0                    inchi_key level1 level2 level3 level4  \
651        651.0  OUKYUETWWIPKQR-UHFFFAOYSA-N    NaN    NaN    NaN    NaN   
1786      1786.0  OUKYUETWWIPKQR-UHFFFAOYSA-N    NaN    NaN    NaN    NaN   

     level5         name        pert_id  \
651     NaN  saracatinib  BRD-U07805514   
1786    NaN  saracatinib  BRD-U0

In [57]:
def common_member(a, b): 
    a_set = set(a) 
    b_set = set(b) 
    if (a_set & b_set): 
        print("Number of common elements: ",len(a_set & b_set))
        print("Total perts: ",len(set(a_set|b_set)))
        # returns all the perts in 2 sets
        return(set(a_set|b_set))
    else: 
        print("No common elements")  

all = list(common_member(atc.name.str.lower(),newatc.name))
all

Number of common elements:  605
Total perts:  1264


[nan,
 'fentiazac',
 'lafutidine',
 'sulfafurazole',
 'secnidazole',
 'dantrolene',
 'physostigmine',
 'flutrimazole',
 'norethisterone',
 'testosterone',
 'fluphenazine',
 'gsk-1292263',
 'piperidolate',
 'cevimeline',
 'enoxolone',
 'mk-3207',
 'tirofiban',
 'famciclovir',
 'olanzapine',
 'resatorvid',
 'pf-04447943',
 'voxtalisib',
 'decitabine',
 'trientine',
 'trimipramine',
 'pha-793887',
 'lidocaine',
 'zofenopril',
 'balsalazide',
 'chlorthalidone',
 'lurasidone',
 'tobramycin',
 'nefazodone',
 'dextromethorphan',
 'raf-265',
 'e-2012',
 'carbamazepine',
 'selumetinib',
 'dutasteride',
 'epirizole',
 'dicloxacillin',
 '2-{[4-(1-methyl-4-pyridin-4-yl-1h-pyrazol-3-yl)phenoxy]methyl}quinoline',
 'pilocarpine',
 'bupivacaine',
 'balaglitazone',
 'glipizide',
 'cefdinir',
 'clofibrate',
 'spebrutinib',
 'fluticasone propionate',
 'triamterene',
 'osi-930',
 'indinavir',
 'betrixaban',
 'doxepin',
 'emedastine',
 'telatinib',
 'tetracaine',
 'teriflunomide',
 'efaproxiral',
 'volaser

In [58]:
wholeatc.name = wholeatc.name.str.lower()

In [59]:
wholeatc.to_csv("wholeatc.csv",index=False)

In [60]:
name_to_id["balaglitazone"]

'BRD-A00147595'

In [61]:
wholeatc.isnull().sum()

Unnamed: 0     768
inchi_key        0
level1        1484
level2        1484
level3        1484
level4        1484
level5        1484
name          1069
pert_id       1069
smiles           0
dtype: int64

In [121]:
df = pd.DataFrame()
n = list(Counter(wholeatc.name).keys())
for i in list(n):
    print(i, end=' ')
    tempdf = wholeatc[(wholeatc.name==i)]
    if len(tempdf)>1:
        test = wholeatc[(wholeatc.name==i) & (wholeatc.smiles!='')]
        print(">1 ",len(test))
        df = df.append(test)
    else:
        print("1 ",len(tempdf))
        df = df.append(tempdf)

flutrimazole >1  1
biperiden >1  1
drospirenone 1  1
bupivacaine >1  1
nitrendipine >1  1
meptazinol >1  1
lofexidine >1  1
alizapride >1  1
mepivacaine >1  1
fulvestrant 1  1
metoprolol >1  1
maraviroc 1  1
chlorphenamine >1  1
bupropion >1  1
docetaxel 1  1
esmolol >1  1
labetalol >1  1
bopindolol >1  1
racecadotril >1  1
doxycycline 1  1
flecainide >1  1
verapamil >1  1
argatroban 1  1
tipranavir 1  1
propranolol >1  1
granisetron 1  1
carvedilol >1  1
pravastatin 1  1
lorazepam >1  1
cefepime 1  1
cisapride 1  1
pomalidomide >1  1
troglitazone >1  1
fluorometholone 1  1
doxazosin >1  1
alaproclate >1  1
milnacipran >1  1
rebamipide >1  1
bucladesine 1  1
indobufen >1  1
butoconazole >1  1
protirelin 1  1
bambuterol >1  1
ibuprofen >1  1
lenalidomide >1  1
phensuximide >1  1
tazobactam 1  1
estriol 1  1
estradiol 1  1
trimipramine >1  1
perhexiline >1  1
mianserin >1  1
ondansetron >1  1
atovaquone 1  1
mecamylamine >1  1
medrysone 1  1
chlormezanone >1  1
praziquantel >1  1
amlodip

mitiglinide >1  1
clomipramine >1  1
vinpocetine >1  1
tivozanib >1  1
fluvoxamine >1  1
baricitinib >1  1
amitriptyline >1  1
empagliflozin >1  1
risperidone >1  1
vortioxetine >1  1
ruxolitinib >1  1
formoterol 1  1
elvitegravir >1  1
dosulepin >1  1
etoricoxib >1  1
lorcaserin >1  1
fluphenazine >1  1
benzylpenicillin >1  1
phenytoin >1  1
oxiconazole >1  1
vemurafenib >1  1
ambroxol >1  1
clofazimine >1  1
fomepizole >1  1
epirubicin 1  1
enzalutamide >1  1
indometacin >1  1
fidaxomicin 1  1
tiapride >1  1
enalapril >1  1
pentoxifylline >1  1
cariprazine >1  1
vilazodone >1  1
rasagiline >1  1
roxithromycin 1  1
dexfenfluramine >1  1
chlorprothixene 1  1
naproxen >1  1
propentofylline >1  1
voglibose 1  1
linopirdine >1  1
tedizolid >1  1
maribavir >1  1
quinidine >1  1
pasireotide >1  1
irbesartan >1  1
tiagabine >1  1
paricalcitol >1  1
idelalisib >1  1
loperamide >1  1
romidepsin >1  1
thiomersal 1  1
emtricitabine >1  1
anagrelide >1  1
dapsone >1  1
venetoclax >1  1
guanethidi

defactinib 1  1
genistein 1  1
camostat 1  1
phenoxymethylpenicillin 1  1
dexniguldipine 1  1
2-methoxyestradiol 1  1
taselisib 1  1
tauroursodeoxycholic acid 1  1
betrixaban 1  1
cc-401 1  1
emapunil 1  1
preladenant 1  1
varlitinib 1  1
zofenopril 1  1
umeclidinium 1  1
antipyrine 1  1
piclamilast 1  1
podofilox 1  1
balapiravir 1  1
teneligliptin 1  1
tecastemizole 1  1
tanzisertib 1  1
methyldopa 1  1
salsalate 1  1
laropiprant 1  1
gsk-2636771 1  1
pilaralisib 1  1
arotinoid acid 1  1
pd-0325901 1  1
pf-04447943 1  1
tolfenamic acid 1  1
canertinib 1  1
meclofenamic acid 1  1
taladegib 1  1
e-2012 1  1
sulfisoxazole 1  1
tirasemtiv 1  1
sarcosine 1  1
siramesine 1  1
gm6001 1  1
hydroxyurea 1  1
biib021 1  1
(4-amino-2-{[1-(methylsulfonyl)piperidin-4-yl]amino}pyrimidin-5-yl)(2,3-difluoro-6-methoxyphenyl)methanone 1  1
tacedinaline 1  1
pictilisib 1  1
ozagrel 1  1
cts-1027 1  1
mk-5108 1  1
ripasudil 1  1
trehalose 1  1
mk-1775 1  1
methimazole 1  1
turofexorate isopropyl 1  1
equ

In [122]:
df.shape

(1264, 10)

In [123]:
cols = list(df)
cols.remove("smiles")
cols.remove("inchi_key")
cols.remove("Unnamed: 0")
df1 = df.dropna(how='any')
df = df1[cols]

In [125]:
df.shape

(686, 7)

In [126]:
df.to_csv("cleaned-atc.csv",index=False)

In [127]:
pd.read_csv("cleaned-atc.csv")

,level1,level2,level3,level4,level5,name,pert_id
0,G,G01,G01A,G01AF,G01AF18,flutrimazole,BRD-A00218260
1,N,N04,N04A,N04AA,N04AA02,biperiden,BRD-A00546892
2,N,N01,N01B,N01BB,N01BB01,bupivacaine,BRD-A01636364
3,C,C08,C08C,C08CA,C08CA08,nitrendipine,BRD-A02006392
4,N,N02,N02A,N02AX,N02AX05,meptazinol,BRD-A02710418
5,N,N07,N07B,N07BC,N07BC04,lofexidine,BRD-A02990301
6,A,A03,A03F,A03FA,A03FA05,alizapride,BRD-A03061970
7,N,N01,N01B,N01BB,N01BB53,mepivacaine,BRD-A03216249
8,C,C07,C07F,C07FB,C07FB02,metoprolol,BRD-A03623303
9,R,R06,R06A,R06AB,R06AB54,chlorphenamine,BRD-A04553218


In [109]:
df

,Unnamed: 0,inchi_key,level1,level2,level3,level4,level5,name,pert_id,smiles
1,1.0,QHMWCHQXCUNUAK-UHFFFAOYSA-N,G,G01,G01A,G01AF,G01AF18,flutrimazole,BRD-A00218260,FC1CCC(CC1)C(C1CCCCC1)(C1CCCCC1F)N1CCNC1
3,3.0,YSXKPIUOCJLQIE-UHFFFAOYSA-N,N,N04,N04A,N04AA,N04AA02,biperiden,BRD-A00546892,OC(CCN1CCCCC1)(C1CC2CC1C=C2)C1CCCCC1
2,NaN,,G,G0,G03,G03AC,G03AC10,drospirenone,BRD-A00938334,
5,5.0,LEBVLXFERQHONN-UHFFFAOYSA-N,N,N01,N01B,N01BB,N01BB01,bupivacaine,BRD-A01636364,CCCCN1CCCCC1C(=O)NC1C(C)CCCC1C
8,8.0,PVHUJELLJLJGLN-UHFFFAOYSA-N,C,C08,C08C,C08CA,C08CA08,nitrendipine,BRD-A02006392,CCOC(=O)C1=C(C)NC(C)=C(C1C1CCCC(C1)[N+]([O-])=...
10,10.0,JLICHNCFTLFZJN-UHFFFAOYSA-N,N,N02,N02A,N02AX,N02AX05,meptazinol,BRD-A02710418,CCC1(CCCCN(C)C1)C1CCCC(O)C1
11,11.0,KSMAGQUYOIHWFS-UHFFFAOYSA-N,N,N07,N07B,N07BC,N07BC04,lofexidine,BRD-A02990301,CC(OC1C(CL)CCCC1CL)C1=NCCN1
12,12.0,KSEYRUGYKHXGFW-UHFFFAOYSA-N,A,A03,A03F,A03FA,A03FA05,alizapride,BRD-A03061970,COC1CC2[NH]NNC2CC1C(=O)NCC1CCCN1CC=C
13,13.0,INWLQCZOYSRPNW-UHFFFAOYSA-N,N,N01,N01B,N01BB,N01BB53,mepivacaine,BRD-A03216249,CN1CCCCC1C(=O)NC1C(C)CCCC1C
9,NaN,,L,L0,L02,L02BA,L02BA03,fulvestrant,BRD-A03249105,
